# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
#import necessary libraries
import pickle
import numpy as np
import random
import matplotlib.pyplot as plt
import random
import cv2
from sklearn.utils import shuffle
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
# Load pickled data



# load pickled files from data folder

training_file = 'data/train.p'
validation_file='data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

#data sets are dictionary entries
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
y_train=np.append(y_train,y_train,axis=0) #double length of y_train, as extra augmented data will be added

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
#store number of data entries in each set.

#Number of training examples (before augmentation)
n_train = len(X_train)

#Number of validation examples
n_validation = len(X_valid)

#Number of testing examples.
n_test = len(X_test)

#Shape of an traffic sign image
image_shape = (len(X_train[0]),len(X_train[0][0]))

#Use set data structure to identify how many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

#print data summary
print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.


fig, axs=plt.subplots(1,3)
_=axs[0].hist(y_train,bins=n_classes)
_=axs[1].hist(y_valid,bins=n_classes)
_=axs[2].hist(y_test,bins=n_classes)

print("training, validation, and testing data sets in order")

Distribution of data seems similar across all 3 data sets. Distribution within the sets is highly irregular.

In [ ]:
#visualize first training image and print RGB value of top left corner
plt.imshow(X_train[0,:,:,:])

In [ ]:
#print 4D array shape for X_train and length of y_train

print(X_train.shape)
print(y_train.shape)

# ----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
#Data is padded first. 2 pixels are padded on all sides of image, changing dimensions to 36x36

X_trainP      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validP = np.pad(X_valid, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_testP       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')


#create new arrays for processed data, which will be in HSV format
h_train=np.zeros((n_train,36,36,3),dtype=np.uint8) #training
h_train_aug=np.zeros((n_train,36,36,3),dtype=np.uint8) #augmented training data
h_valid=np.zeros((n_validation,36,36,3),dtype=np.uint8) #validation data
h_test=np.zeros((n_test,36,36,3),dtype=np.uint8) #test data
print('arrays created')

#for all training data
for i in range(n_train):
    im=X_trainP[i] #store image
    h_train[i,:,:,:]=cv2.cvtColor(im,cv2.COLOR_RGB2HSV) #convert RGB to HSV
    
    #data augmentation step: rotate up to +/-7 degrees
    #with 2px padding, cap of 7 degrees means no information is lost
    M=cv2.getRotationMatrix2D((18,18),14*random.random()-7,1).astype(float) #create transformation matrix
    dst=cv2.warpAffine(h_train[i,:,:,:],M,(36,36)) #rotate by up to 7 degrees
    dst=np.resize(dst,(1,36,36,3)) #add 4th dimension
    h_train_aug[i,:,:,:]=dst #add to augmentation matrix
    
h_train=np.append(h_train,h_train_aug,axis=0) #add augmentation matrix to standard matrix
preRot=np.copy(h_train[0,:,:,:]) #store copy of pre-rotated image before normalization
postRot=np.copy(h_train[n_train,:,:,:]) #same but post-rotation
h_train=h_train.astype(float) #cast as float
h_train[:,:,:,0]=np.subtract(np.divide(h_train[:,:,:,0],90.),1.) #normalize hue, saturation, value
h_train[:,:,:,1]=np.subtract(np.divide(h_train[:,:,:,1],128.),1.)
h_train[:,:,:,2]=np.subtract(np.divide(h_train[:,:,:,2],128.),1.)
print('training done')
for i in range(n_validation): #repeat for validation and test data, with exception of augmentation
    h_valid[i,:,:,:]=cv2.cvtColor(X_validP[i],cv2.COLOR_RGB2HSV)
h_valid=h_valid.astype(float)
h_valid[:,:,:,0]=np.subtract(np.divide(h_valid[:,:,:,0],90.),1.)
h_valid[:,:,:,1]=np.subtract(np.divide(h_valid[:,:,:,1],128.),1.)
h_valid[:,:,:,2]=np.subtract(np.divide(h_valid[:,:,:,2],128.),1.)
print('validation done')
for i in range(n_test):
    h_test[i,:,:,:]=cv2.cvtColor(X_testP[i],cv2.COLOR_RGB2HSV)
h_test=h_test.astype(float)
h_test[:,:,:,0]=np.subtract(np.divide(h_test[:,:,:,0],90.0),1.0)
h_test[:,:,:,1]=np.subtract(np.divide(h_test[:,:,:,1],128.0),1.0)
h_test[:,:,:,2]=np.subtract(np.divide(h_test[:,:,:,2],128.0),1.0)


print('test done')

In [ ]:
preRot=cv2.cvtColor(preRot,cv2.COLOR_HSV2RGB) #convert back to RGB

plt.imshow(preRot) #show prerotated image to verify no changes

#print first non-padded pixel
print(preRot[2,2,:])

In [ ]:
#repeat for post-rotation image

postRot=cv2.cvtColor(postRot,cv2.COLOR_HSV2RGB)

plt.imshow(postRot)

print(postRot[2,2,:])

In [ ]:
print(np.max(h_test[1,:,:,:]))
print(np.min(h_test[1,:,:,:]))

#verify normalization, results should be near 1 and -1

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import tensorflow as tf
from tensorflow.contrib.layers import flatten

EPOCHS=15 #increase number of epochs
BATCH_SIZE=32 #lower batch size

def LeNet(x,keep):
    
    #3 convolution layers with 4 fully connected layers, ReLu activation functions, 2x2 max pooling between
    #convolution and 1st fully connected layer
    
    mu=0 #initialize mu and sigma for gaussian distribution use
    sigma=0.1
    keep_prob=keep #keep probability for dropout
    
    conv1depth=256 #set depth of convolutional layers
    conv2depth=128
    conv3depth=64
    flatten1=12*12*conv3depth
    
    fc1size=200
    fc2size=100
    fc3size=75
    fc4size=43
    
    #layer 1 convolution
    
    conv1_weights= tf.Variable(tf.truncated_normal(shape=(5,5,3,conv1depth), mean=mu, stddev=sigma))
    conv1_biases=tf.Variable(tf.zeros(conv1depth))
    conv1=tf.nn.conv2d(x,conv1_weights,strides=[1,1,1,1],padding='VALID')+conv1_biases
    
    #activation function 1
    conv1=tf.nn.relu(conv1)
    conv1=tf.nn.dropout(conv1,keep_prob) #dropout 1
    
    #Layer 2 convolutional
    
    conv2_weights=tf.Variable(tf.truncated_normal(shape=(5,5,conv1depth,conv2depth),mean=mu,stddev=sigma))
    conv2_biases=tf.Variable(tf.zeros(conv2depth))
    conv2=tf.nn.conv2d(conv1,conv2_weights,strides=[1,1,1,1],padding='VALID')+conv2_biases
    
    #Activation function 2
    
    conv2=tf.nn.relu(conv2)
    
    conv2=tf.nn.dropout(conv2,keep_prob) #dropout 2
    
    #layer 3 convolutional
    
    conv3_weights=tf.Variable(tf.truncated_normal(shape=(5,5,conv2depth,conv3depth),mean=mu,stddev=sigma))
    conv3_biases=tf.Variable(tf.zeros(conv3depth))
    conv3=tf.nn.conv2d(conv2,conv3_weights,strides=[1,1,1,1],padding='VALID')+conv3_biases
    
    #Activation function 3
    
    conv3=tf.nn.relu(conv3)
    
    conv3=tf.nn.dropout(conv3,keep_prob) #dropout 3
    
    #pooling
    conv3=tf.nn.max_pool(conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    
    
    #flatten
    fc0=flatten(conv3)
    
    # Layer FC1:
    fc1_weights = tf.Variable(tf.truncated_normal(shape=(flatten1, fc1size), mean = mu, stddev = sigma))
    fc1_biases = tf.Variable(tf.zeros(fc1size))
    fc1   = tf.matmul(fc0, fc1_weights) + fc1_biases
    
    # Activation.
    fc1    = tf.nn.relu(fc1)

    # FC2
    fc2_weights  = tf.Variable(tf.truncated_normal(shape=(fc1size, fc2size), mean = mu, stddev = sigma))
    fc2_biases  = tf.Variable(tf.zeros(fc2size))
    fc2    = tf.matmul(fc1, fc2_weights) + fc2_biases
    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer FC3
    fc3_weights  = tf.Variable(tf.truncated_normal(shape=(fc2size, fc3size), mean = mu, stddev = sigma))
    fc3_biases  = tf.Variable(tf.zeros(fc3size))
    fc3    = tf.matmul(fc2, fc3_weights) + fc3_biases
    fc3    = tf.nn.relu(fc3)
    
    # Layer FC4
    
    fc4_weights  = tf.Variable(tf.truncated_normal(shape=(fc3size, fc4size), mean = mu, stddev = sigma))
    fc4_biases  = tf.Variable(tf.zeros(fc4size))
    logits = tf.matmul(fc3, fc4_weights) + fc4_biases
    
    return logits



### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train model

x = tf.placeholder(tf.float32, (None, 36,36, 3)) #3-color channel image
y = tf.placeholder(tf.int32, (None,)) #y label
keep = tf.placeholder(tf.float32,shape=()) #dropout keep probability
one_hot_y = tf.one_hot(y, n_classes) #one-hot y-label

rate = 0.001 #learning rate

logits = LeNet(x,keep)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

#evaluate accuracy of model
def evaluate(X_data, y_data, keepNum):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep: keepNum})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


#evaluate but also return top 5 probabilities 
def evaluate2(X_data, y_data, keepNum, topNum=5):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        return sess.run(tf.nn.top_k(tf.nn.softmax(sess.run(logits, feed_dict={x: batch_x, y: batch_y, keep: keepNum})),k=topNum))

In [ ]:
with tf.Session() as sess: #start session
    sess.run(tf.global_variables_initializer()) #initialize variables
    num_examples = len(h_train) #number of examples is length of training variable
    
    print("Training...")
    print()
    
    accuracyMax=0
    for i in range(EPOCHS):
        h_train, y_train = shuffle(h_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = h_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep:0.65}) #run with 65% dropout
            
        validation_accuracy = evaluate(h_valid, y_valid, 1) #calculate percentage with validation set and no dropout
        
        
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        if validation_accuracy>accuracyMax+30.0/n_validation: #if model is an improvement
            saver.save(sess, './lenet') #save model
            print("Model saved")
            accuracyMax=validation_accuracy
        print()
        
    

In [ ]:
#calcluate training accuracy

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    train_accuracy = evaluate(h_train, y_train, 1.0) #evaluate with training data and no dropout
    print("Train Accuracy = {:.3f}".format(train_accuracy))

In [ ]:
#evaluate with test data

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(h_test, y_test, 1.0) #evaluate with test data and no dropout
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images found online.

n_web=5 #number of images downloaded from web

xWeb=np.zeros((n_web,32,32,3)) #images are 32x32x3
yWeb=np.zeros(n_web) #create label variable
idNum=[4, 13, 14, 32, 33] #actual id numbers of images

for i in range(n_web):
    xWeb[i,:,:,:]=plt.imread('data/id'+str(idNum[i])+'.png')[:,:,0:3] #load image from file
    yWeb[i]=idNum[i] #add actual label to y array
    
#pre processing similar to above
    
#xWeb is currently scale 0-1, multiply by 255
xWeb=np.multiply(xWeb,255).astype(np.uint8)

#pad data
xWebP      = np.pad(xWeb, ((0,0),(2,2),(2,2),(0,0)), 'constant')

#prepare processed image 4d array
h_web=np.zeros((n_web,36,36,3),dtype=np.uint8)

#for all web images
for i in range(n_web):
    h_web[i,:,:,:]=cv2.cvtColor(xWebP[i,:,:,0:3],cv2.COLOR_RGB2HSV) #convert to HSV
    
h_web=h_web.astype(float) #convert to float


#normalize channels
h_web[:,:,:,0]=np.subtract(np.divide(h_web[:,:,:,0],90.0),1.0)
h_web[:,:,:,1]=np.subtract(np.divide(h_web[:,:,:,1],128.0),1.0)
h_web[:,:,:,2]=np.subtract(np.divide(h_web[:,:,:,2],128.0),1.0)

print('x_label size for web images: ' + str(h_web.shape))
print('x_label size for test images: ' + str(h_test.shape))
print('y_label size for web images: ' + str(yWeb.shape))
print('y_label size for test images: ' + str(y_test.shape))

Data shapes appear to be correct

### Predict the Sign Type for Each Image

In [ ]:
#Test web images

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    (sM, ind) =evaluate2(h_web, yWeb, 1.0, topNum=1)
    
    print(ind) #print most likely index

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 


    
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(h_web, yWeb, 1.0) #no dropout
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image).

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web.

#Test web images to get top 5

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    (sM, ind) =evaluate2(h_web, yWeb, 1.0)
    print('Top 5 softmax probabilities for each image:')
    print(sM)
    print()
    print('Labels for each softmax probability')
    print(ind)